<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Image-Preprocess-and-Model-Block" data-toc-modified-id="Image-Preprocess-and-Model-Block-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Image Preprocess and Model Block</a></span></li><li><span><a href="#Res-Block(option)" data-toc-modified-id="Res-Block(option)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Res Block(option)</a></span></li><li><span><a href="#Definite-GAN" data-toc-modified-id="Definite-GAN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Definite GAN</a></span></li><li><span><a href="#Read-and-Generate-Data" data-toc-modified-id="Read-and-Generate-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Read and Generate Data</a></span></li><li><span><a href="#Model-Build-and-Training" data-toc-modified-id="Model-Build-and-Training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model Build and Training</a></span></li><li><span><a href="#Spectral-Normalization" data-toc-modified-id="Spectral-Normalization-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Spectral Normalization</a></span></li></ul></div>

## Image Preprocess and Model Block

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt


from PIL import Image
import cv2
from math import floor
import time

from functools import partial,update_wrapper
from functools import wraps

from random import random

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation,Lambda,GlobalMaxPooling2D,Concatenate,GlobalAveragePooling2D
from keras.layers import LeakyReLU,Dot,Add,Layer
from keras.layers import Reshape, UpSampling2D, Flatten, Input, add, Lambda, concatenate, LeakyReLU, multiply
from keras.layers import Conv2D, Dense, AveragePooling2D, Activation, Cropping2D, Dropout, average
from keras.layers import UpSampling2D, Conv2D,Conv2DTranspose,Multiply
from keras.layers.merge import _Merge

from keras.models import Sequential, Model, model_from_json, Model
from keras.optimizers import RMSprop,Adam
from keras import metrics
import keras.backend as K
from keras.initializers import VarianceScaling

im_size = 128 # image size
resize=200 # image resize before cut
latent_size = 128 # laten dim
BATCH_SIZE = 45 # batch size
channels = 3 # image channel
d = 32 # attention image size
att_fil_div = 8 # attention filter division
adam_eps=1e-8 # optimizer adam parameter
cha = 16 # base convolution layer filter number

# lr = 0.0001
# enc_lr = lr*10
# gen_lr = lr*1
# dis_lr = lr*4

def noise(n):
    return np.random.normal(0.0, 1.0, size = [n, latent_size])

def hinge_d(y_true, y_pred):
    return K.mean(K.relu(1.0 - (y_true * y_pred)))

def w_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

class Gamma(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Gamma, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1,),
                                      initializer='zeros',
                                      trainable=True)
        super(Gamma, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return x*self.kernel

    def compute_output_shape(self, input_shape):
        return (input_shape)
    
def self_att_block(inp,fil):
    n=d**2
    query_conv = Conv2D(fil//att_fil_div, 1)(inp)
    key_conv = Conv2D(fil//att_fil_div, 1)(inp)
    value_conv = Conv2D(fil, 1)(inp)
    
    query_conv = Reshape((n,-1))(query_conv)
    key_conv = Reshape((n,-1))(key_conv)
    attention = Dot(-1)([query_conv,key_conv])
    attention = Activation('softmax')(attention)
    
    value_conv = Reshape((n,-1))(value_conv)
    value_conv = Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(value_conv)
    out = Dot(-1)([attention,value_conv])
    out = Reshape((d,d,-1))(out)
    
    out = Gamma((1,))(out)
    out = Add()([inp,out])
    
    return out

def self_att_block_sn(inp,fil):
    n=d**2
    query_conv = ConvSN2D(fil//att_fil_div, 1)(inp)
    key_conv = ConvSN2D(fil//att_fil_div, 1)(inp)
    value_conv = ConvSN2D(fil, 1)(inp)
    
    query_conv = Reshape((n,-1))(query_conv)
    key_conv = Reshape((n,-1))(key_conv)
    attention = Dot(-1)([query_conv,key_conv])
    attention = Activation('softmax')(attention)
    
    value_conv = Reshape((n,-1))(value_conv)
    value_conv = Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(value_conv)
    out = Dot(-1)([attention,value_conv])
    out = Reshape((d,d,-1))(out)
    out = Gamma((1,))(out)
    out = Add()([inp,out])
    
    return out

def g_block(inp, fil):
    out = ConvSN2DTranspose(fil, kernel_size=4, strides=2, padding = 'same', kernel_initializer = 'he_normal')(inp)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    return out

def d_block(inp, fil):
    out = Conv2D(filters = fil, kernel_size=4, strides=2, padding = 'same', kernel_initializer = 'he_normal')(inp)
    out = BatchNormalization()(out)
    out = LeakyReLU(0.1)(out)
    return out

def dis_block(inp, fil):
    out = ConvSN2D(fil, kernel_size=4, strides=2, padding = 'same', kernel_initializer = 'he_normal')(inp)
    out = LeakyReLU(0.1)(out)
    return out

## Res Block(option)

In [ ]:
def g_block(inp, fil, u = True):

    if u:
        out = UpSampling2D(interpolation = 'bilinear')(inp)
    else:
        out = Activation('linear')(inp)

    skip = ConvSN2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(out)

    out = ConvSN2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = LeakyReLU(0.2)(out)
#     out = BatchNormalization(trainable=False)(out)
    out = BatchNormalization()(out)

    out = ConvSN2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = LeakyReLU(0.2)(out)
#     out = BatchNormalization(trainable=False)(out)
    out = BatchNormalization()(out)

    out = ConvSN2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = add([out, skip])
    out = LeakyReLU(0.2)(out)
#     out = BatchNormalization(trainable=False)(out)
    out = BatchNormalization()(out)

    return out

def d_block(inp, fil, p = True):

    skip = Conv2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(inp)

    out = Conv2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(inp)
    out = LeakyReLU(0.2)(out)
#     out = BatchNormalization(trainable=False)(out)
    out = BatchNormalization()(out)

    out = Conv2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = LeakyReLU(0.2)(out)
#     out = BatchNormalization(trainable=False)(out)
    out = BatchNormalization()(out)


    out = Conv2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(out)

    out = add([out, skip])
    out = LeakyReLU(0.2)(out)
#     out = BatchNormalization(trainable=False)(out)
    out = BatchNormalization()(out)

    if p:
        out = AveragePooling2D()(out)

    return out

def dis_block(inp, fil, p = True):

    skip = ConvSN2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(inp)

    out = ConvSN2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(inp)
    out = LeakyReLU(0.2)(out)

    out = ConvSN2D(filters = fil, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')(out)
    out = LeakyReLU(0.2)(out)

    out = ConvSN2D(fil, 1, padding = 'same', kernel_initializer = 'he_normal')(out)

    out = add([out, skip])
    out = LeakyReLU(0.2)(out)

    if p:
        out = AveragePooling2D()(out)

    return out

## Definite GAN

In [3]:
class GAN(object):

    def __init__(self, steps = 1, lr = 0.0001, decay = 0.00001):

        #Models
        self.D = None
        self.E = self.encoder()
        self.G = self.generator()

        self.GE = self.generator()
        self.EE = self.encoder()

        self.DM = None
        self.AM = None

        #Config
        self.LR = lr
        self.steps = steps
        self.beta = 0.999

        #Init Models
        self.discriminator()
        
#         self.E.load_weights('sn-sa-bigan-new-128-sfat32-no-res/36000epoch_encoder.h5')
#         self.G.load_weights('sn-sa-bigan-new-128-sfat32-no-res/36000epoch_generator.h5')
#         self.D.load_weights('sn-sa-bigan-new-128-sfat32-no-res/36000epoch_critic_o.h5')
#         self.generator()
#         self.encoder()
#         self.EE = model_from_json(self.E.to_json())
#         self.EE.set_weights(self.E.get_weights())
#         self.GE = model_from_json(self.G.to_json())
#         self.GE.set_weights(self.G.get_weights())

    def discriminator(self):

        if self.D:
            return self.D

        inp = Input(shape = [im_size, im_size, channels])
        inpl = Input(shape = [latent_size])

        #Latent input
        l = DenseSN(512, kernel_initializer = 'he_normal')(inpl)
        l = LeakyReLU(0.2)(l)
        l = DenseSN(512, kernel_initializer = 'he_normal')(l)
        l = LeakyReLU(0.2)(l)
        l = DenseSN(512, kernel_initializer = 'he_normal')(l)
        l = LeakyReLU(0.2)(l)
        sz = Dense(1, kernel_initializer = 'he_normal')(l)

        
        x = dis_block(inp, 4 * cha)   #64
        x = dis_block(x, 8 * cha)  #32
        x = self_att_block_sn(x, 8 * cha)
        x = dis_block(x, 8 * cha)  #16
        x = dis_block(x, 16 * cha)  #8
        x = dis_block(x, 16* cha)  #4

        x = Flatten()(x)
        
        sx = Dense(1, kernel_initializer = 'he_normal')(x)
        xz = concatenate([x, l])
        xz = DenseSN(16 * cha, kernel_initializer = 'he_normal')(xz)
        xz = LeakyReLU(0.2)(xz)

        sxz = Dense(1, kernel_initializer = 'he_normal')(xz)
        
        s = Add()([sxz,sx,sz])

        self.D = Model(inputs = [inp, inpl], outputs = s)

        return self.D

    def generator(self):

#         if self.G:
#             return self.G

        #Inputs
        inp = Input(shape = [latent_size])

        #Latent

        #Actual Model
        x = Dense(4*4*16*cha, kernel_initializer = 'he_normal')(inp)
        x = Reshape([4, 4, 16*cha])(x)

        x = g_block(x, 16 * cha)  #4
        x = g_block(x, 16 * cha)  #8
        x = g_block(x, 8 * cha)  #16
        x = self_att_block_sn(x, 8 * cha)
        x = g_block(x, 8 * cha)   #64
        x = g_block(x, 4 * cha)   #128

        x = Conv2D(filters = channels, kernel_size = 1, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(x)
        
        return Model(inputs = inp, outputs = x)
#         self.G = Model(inputs = inp, outputs = x)
#         return self.G

    def encoder(self):

#         if self.E:
#             return self.E

        inp = Input(shape = [im_size, im_size, channels])

        x = d_block(inp, 4 * cha)   #64
        x = d_block(x, 8 * cha)   #32
        x = self_att_block(x, 8 * cha)
        x = d_block(x, 8 * cha)   #16
        x = d_block(x, 16 * cha)  #8
        x = d_block(x, 16 * cha)  #4
#         x = d_block(x, 16 * cha, p = False)  #4


        x = Flatten()(x)

#         x = Dense(16 * cha, kernel_initializer = 'he_normal')(x)
#         x = LeakyReLU(0.2)(x)

        x = Dense(latent_size, kernel_initializer = 'he_normal', bias_initializer = 'zeros')(x)
        return Model(inputs = inp, outputs = x)
#         self.E = Model(inputs = inp, outputs = x)
#         return self.E


    def AdModel(self):

        #D does not update
        self.D.trainable = False
        for layer in self.D.layers:
            layer.trainable = False

        #G does update
        self.G.trainable = True
        for layer in self.G.layers:
            layer.trainable = True

        #E does update
        self.E.trainable = True
        for layer in self.E.layers:
            layer.trainable = True

        # Fake Latent / Real Image
        ri = Input(shape = [im_size, im_size, channels])

        er = self.E(ri)
        dr = self.D([ri, er])

        # Real Latent / Fake Image
        gi = Input(shape = [latent_size])

        gf = self.G(gi)
        df = self.D([gf, gi])

        self.AM = Model(inputs = [ri, gi], outputs = [dr, df])
        self.E_tr = Model(inputs = ri, outputs = dr)
        self.G_tr = Model(inputs = gi, outputs = df)
#         self.E_tr = multi_gpu_model(Model(inputs = ri, outputs = dr),2)
#         self.G_tr = multi_gpu_model(Model(inputs = gi, outputs = df),2)


        self.E_tr.compile(optimizer = Adam(self.LR*1.5, beta_1 = 0, beta_2 = 0.999,epsilon=adam_eps), loss = w_loss)
        self.G_tr.compile(optimizer = Adam(self.LR*1.5, beta_1 = 0, beta_2 = 0.999,epsilon=adam_eps), loss = w_loss)
#         self.AM.compile(optimizer = Adam(self.LR, beta_1 = 0, beta_2 = 0.999,epsilon=adam_eps), loss = [w_loss, w_loss])

        return self.E_tr, self.G_tr

    def DisModel(self):

        #D does update
        self.D.trainable = True
        for layer in self.D.layers:
            layer.trainable = True

        #G does not update
        self.G.trainable = False
        for layer in self.G.layers:
            layer.trainable = False

        #E does update
        self.E.trainable = False
        for layer in self.E.layers:
            layer.trainable = False

        # Fake Latent / Real Image
        ri = Input(shape = [im_size, im_size, channels])

        er = self.E(ri)
        dr = self.D([ri, er])

        # Real Latent / Fake Image
        gi = Input(shape = [latent_size])

        gf = self.G(gi)
        df = self.D([gf, gi])

        self.DM = Model(inputs = [ri, gi], outputs = [dr, df])
#         self.DM = multi_gpu_model(Model(inputs = [ri, gi], outputs = [dr, df]),2)
        self.DM.compile(optimizer = Adam(self.LR*1, beta_1 = 0, beta_2 = 0.999,epsilon=adam_eps),
                        loss=[hinge_d, hinge_d])
        return self.DM
    
    def EMA(self):

        start = time.clock()

        for i in range(len(self.G.layers)):
            up_weight = self.G.layers[i].get_weights()
            old_weight = self.GE.layers[i].get_weights()
            new_weight = []
            for j in range(len(up_weight)):
                new_weight.append(old_weight[j] * self.beta + (1-self.beta) * up_weight[j])
            self.GE.layers[i].set_weights(new_weight)

        for i in range(len(self.E.layers)):
            up_weight = self.E.layers[i].get_weights()
            old_weight = self.EE.layers[i].get_weights()
            new_weight = []
            for j in range(len(up_weight)):
                new_weight.append(old_weight[j] * self.beta + (1-self.beta) * up_weight[j])
            self.EE.layers[i].set_weights(new_weight)

#         print("Moved Average. " + str(time.clock() - start) + "s")

    def MAinit(self):
        self.EE.set_weights(self.E.get_weights())
        self.GE.set_weights(self.G.get_weights())

## Read and Generate Data 

In [6]:
path_1 = ''
df_have_fc_tr = pd.read_csv(path_1+'df_have_fc_tr.csv',index_col='index')
df_have_fc_val = pd.read_csv(path_1+'df_have_fc_val.csv',index_col='index')
fix_image = pd.read_csv(path_1+'fix_image.csv')
dir_f = ''
fix_image.tr = dir_f+fix_image.tr
fix_image.val = dir_f+fix_image.val
df_have_fc_tr.index = dir_f+df_have_fc_tr.index
df_have_fc_val.index = dir_f+df_have_fc_val.index

def ImageProcessing(path,resize=resize,cut_size=im_size,random_cut=True):
    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # 把大小調到 400 * 400
    img = cv2.resize(img, (resize, resize), interpolation=cv2.INTER_CUBIC)
    (h, w) = img.shape[:2]
    # 把尺裁掉
    h = int(h*(9/10))
    img = img[:h]
    # 隨機水平翻轉
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    coin = np.random.randint(2)
    if coin == 0:
        img = cv2.flip(img, 1)
    # 隨機裁減圖片
    if random_cut==True:
        h_s = np.random.randint(h-cut_size)
        w_s = np.random.randint(w-cut_size)
        img = img[h_s:h_s+cut_size,w_s:w_s+cut_size]
    else:
        ch,cw = img.shape[0]//2, img.shape[1]//2
        img = img[ch-cut_size//2:ch+cut_size//2,cw-cut_size//2:cw+cut_size//2]
    img = img.astype('float')
    img-=127.5; img/=127.5

    return img

def gen_data(index_list):
    res = []
    for i in index_list:
        res.append(ImageProcessing(i))
    return np.r_[res]
def gen_data_fix(index_list):
    res = []
    for i in index_list:
        res.append(ImageProcessing_fix(i))
    return np.r_[res]    
def balance_data_gen(bs = BATCH_SIZE//15):
    img_name = []
    col = df_have_fc_tr.Layer_1.unique()
    for i in col:
        col_idx = df_have_fc_tr[df_have_fc_tr.Layer_1==i].index
        img_name.append(np.random.choice(col_idx,size=bs,replace=True))
    img_name = np.concatenate(img_name)
    x_img = gen_data(img_name)
#         print(img_name)
    return x_img

## Model Build and Training

In [8]:
from tqdm import tqdm_notebook
from keras.datasets import cifar10
class BiGAN(object):

    def __init__(self, steps = 1, lr = 0.0001, decay = 0.00001, silent = True):

        self.GAN = GAN(steps = steps, lr = lr, decay = decay)
        self.DisModel = self.GAN.DisModel()
#         self.AdModel 
        self.E_tr, self.G_tr = self.GAN.AdModel()

        
        self.latent_dim = latent_size
        self.batch_size=BATCH_SIZE
        self.silent = silent

        #Train Generator to be in the middle, not all the way at real. Apparently works better??
        self.ones = np.ones((BATCH_SIZE, 1), dtype=np.float32)
        self.zeros = np.zeros((BATCH_SIZE, 1), dtype=np.float32)
        self.nones = -self.ones

    
    def train_fabric(self, epochs_last_time, epochs,dir1='sngan_plot_qq_1/',train_process_name='train_process'):
        train_process = pd.DataFrame()
#         train_process = pd.read_csv('sn-sa-bigan-new-128-sfat32-no-res/train_process2.csv')
        if not os.path.isdir(dir1):
            os.mkdir(dir1)

            
        fixed_real_img_idx_tr = fix_image.tr
        fixed_real_img_idx_val = fix_image.val
        fixed_real_img_tr = gen_data(fixed_real_img_idx_tr)
        fixed_real_img_val = gen_data_fix(fixed_real_img_idx_val)                

        

        fixed_noise = np.random.normal(size=(40,self.latent_dim))
        dummy_y = np.zeros((self.batch_size, 1))

        for epoch in tqdm_notebook(range(epochs_last_time,epochs)):
        
            # 真實圖片跟雜訊
            
            imgs = balance_data_gen()
#             imgs = X_train[np.random.choice(tr_idx,size=self.batch_size,replace=False)]
            noise = np.random.normal(0, 1, (self.batch_size, self.latent_dim))

            
        
            train_data = [imgs,noise]
            d_loss = self.DisModel.train_on_batch(train_data, [self.ones, self.nones])

            ge_loss = self.E_tr.train_on_batch(imgs, self.ones)
            gg_loss = self.G_tr.train_on_batch(noise, self.nones)
#             g_loss = self.AdModel.train_on_batch(train_data, [self.ones, self.nones])

        
        
        
            embed = self.GAN.E.predict(imgs)
            mean_ = embed.mean()
            std_ = embed.std()
            std_2 = embed.std(axis=0).mean()
            
#             print('------epoch',epoch,'------')
#             print('|d_loss: ',d_loss,'\n|g_loss_noise: ',[ge_loss+gg_loss,ge_loss,gg_loss],'mean/std',[mean_,std_])
#             print(embed.std(axis=0).shape)
            
            train_process = self.save_train_process(train_process,epoch,d_loss, [ge_loss+gg_loss,ge_loss,gg_loss],
                                                    [mean_,std_,std_2])            
            
            path = dir1+train_process_name+'.csv'
            train_process.to_csv(path)
            
            if epoch%10==0:
                
                self.GAN.EMA()

            if epoch == 20000:
                self.GAN.MAinit()
            
            if epoch%500==0:
#                 recon_tr = self.generator.predict(self.encoder.predict(fixed_real_img_tr)[0])
#                 recon_val = self.generator.predict(self.encoder.predict(fixed_real_img_val)[0])
                recon_tr = self.GAN.G.predict(self.GAN.E.predict(fixed_real_img_tr))
                recon_val = self.GAN.G.predict(self.GAN.E.predict(fixed_real_img_val))
                fake_fix = self.GAN.G.predict(fixed_noise)
                plt.figure(figsize=(60,40))        
                for i in range(1,31):
                    if i<=15:
                        plt.subplot(10,15,i)
                        plt.imshow((np.squeeze(fixed_real_img_tr[i-1])+1)/2)
                        plt.subplot(10,15,i+15)
                        plt.imshow((np.squeeze(recon_tr[i-1])+1)/2)
                    else:
                        plt.subplot(10,15,i+15)
                        plt.imshow((np.squeeze(fixed_real_img_tr[i-1])+1)/2)
                        plt.subplot(10,15,i+30)
                        plt.imshow((np.squeeze(recon_tr[i-1])+1)/2)


                for i in range(61,91):
                    if i <=75:
                        plt.subplot(10,15,i)
                        plt.imshow((np.squeeze(fixed_real_img_val[i-61])+1)/2)
                        plt.subplot(10,15,i+15)
                        plt.imshow((np.squeeze(recon_val[i-61])+1)/2)
                    else:
                        plt.subplot(10,15,i+15)
                        plt.imshow((np.squeeze(fixed_real_img_val[i-61])+1)/2)
                        plt.subplot(10,15,i+30)
                        plt.imshow((np.squeeze(recon_val[i-61])+1)/2)
                    
                for i in range(121,151):
                    plt.subplot(10,15,i)
                    plt.imshow((np.squeeze(fake_fix[i-121])+1)/2)
                file = dir1+'20822iter_'+str(epoch)+'.jpg'
                plt.savefig(file)
                plt.show()
                plt.close()
                plt.figure(figsize=(10,5))
                plt.hist(embed.ravel(),bins=100)
                file = dir1+'20822iter_dist_'+str(epoch)+'.jpg'
                plt.savefig(file)
                plt.show()
                plt.close()         
                
                self.GAN.E.save_weights(dir1+str(epoch)+'epoch_encoder.h5')
                self.GAN.G.save_weights(dir1+str(epoch)+'epoch_generator.h5')
                self.GAN.D.save_weights(dir1+str(epoch)+'epoch_critic_o.h5')
                train_process = train_process[['d_loss', 'g_loss', 'd_real_score', 'd_fake_sore',
                       'g_real_score', 'g_fake_score', 'mean', 'std','std_ef']]
                col = train_process.columns                
                display(train_process.iloc[::-1][:5])
                plt.figure(figsize=(20,8))
                for i in col:
                    plt.plot(train_process[i])
                plt.legend(col)
                plt.show()
                plt.close() 
                
                col = ['d_loss', 'g_loss']             
                display(train_process.iloc[::-1][:5])
                plt.figure(figsize=(20,8))
                for i in col:
                    plt.plot(train_process[i])
                plt.legend(col)
                plt.show()
                plt.close() 
                
    def save_train_process(self,train_process,epoch,d_loss, g_loss,sta):
        train_process.loc[epoch,'d_loss']=d_loss[0]
        train_process.loc[epoch,'d_real_score']=1-d_loss[1]
        train_process.loc[epoch,'d_fake_sore']=d_loss[2]-1
#         train_process.loc[epoch,'d_wgp_f1_loss']=d_loss[3]
#         train_process.loc[epoch,'d_wgp_loss']=d_loss[4]
        
        train_process.loc[epoch,'g_loss']=g_loss[0]
        train_process.loc[epoch,'g_real_score']=g_loss[1]
        train_process.loc[epoch,'g_fake_score']=-g_loss[2]
        train_process.loc[epoch,'mean']=sta[0]
        train_process.loc[epoch,'std']=sta[1]
        train_process.loc[epoch,'std_ef']=sta[2]
        
        return train_process

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
sngan = BiGAN(lr=0.0001)
# sngan.critic_o.summary()
dir_name='sn-sa-bigan-new-128-sfat32-use-res-3-conv-bn-act-gpuno2'+'/'
sngan.train_fabric(epochs_last_time=0,epochs=1000000,dir1=dir_name,train_process_name='train_process2')

## Spectral Normalization

In [ ]:
from keras import backend as K
from keras.engine import *
from keras.legacy import interfaces
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils.generic_utils import func_dump
from keras.utils.generic_utils import func_load
from keras.utils.generic_utils import deserialize_keras_object
from keras.utils.generic_utils import has_arg
from keras.utils import conv_utils
from keras.legacy import interfaces
from keras.layers import Dense, Conv1D, Conv2D, Conv3D, Conv2DTranspose, Embedding
import tensorflow as tf

class DenseSN(Dense):
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                                 initializer=initializers.RandomNormal(0, 1),
                                 name='sn',
                                 trainable=False)
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True
        
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                 W_bar = K.reshape(W_bar, W_shape)  
        output = K.dot(inputs, W_bar)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output 
        
class _ConvSN(Layer):

    def __init__(self, rank,
                 filters,
                 kernel_size,
                 strides=1,
                 padding='valid',
                 data_format=None,
                 dilation_rate=1,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 spectral_normalization=True,
                 **kwargs):
        super(_ConvSN, self).__init__(**kwargs)
        self.rank = rank
        self.filters = filters
        self.kernel_size = conv_utils.normalize_tuple(kernel_size, rank, 'kernel_size')
        self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
        self.padding = conv_utils.normalize_padding(padding)
        self.data_format = conv_utils.normalize_data_format(data_format)
        self.dilation_rate = conv_utils.normalize_tuple(dilation_rate, rank, 'dilation_rate')
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(ndim=self.rank + 2)
        self.spectral_normalization = spectral_normalization
        self.u = None
        
    def _l2normalize(self, v, eps=1e-12):
        return v / (K.sum(v ** 2) ** 0.5 + eps)
    
    def power_iteration(self, u, W):
        '''
        Accroding the paper, we only need to do power iteration one time.
        '''
        v = self._l2normalize(K.dot(u, K.transpose(W)))
        u = self._l2normalize(K.dot(v, W))
        return u, v
    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        #Spectral Normalization
        if self.spectral_normalization:
            self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                                     initializer=initializers.RandomNormal(0, 1),
                                     name='sn',
                                     trainable=False)
        
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True

    def call(self, inputs):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        
        if self.spectral_normalization:
            W_shape = self.kernel.shape.as_list()
            #Flatten the Tensor
            W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
            _u, _v = power_iteration(W_reshaped, self.u)
            #Calculate Sigma
            sigma=K.dot(_v, W_reshaped)
            sigma=K.dot(sigma, K.transpose(_u))
            #normalize it
            W_bar = W_reshaped / sigma
            #reshape weight tensor
            if training in {0, False}:
                W_bar = K.reshape(W_bar, W_shape)
            else:
                with tf.control_dependencies([self.u.assign(_u)]):
                    W_bar = K.reshape(W_bar, W_shape)

            #update weitht
            self.kernel = W_bar
        
        if self.rank == 1:
            outputs = K.conv1d(
                inputs,
                self.kernel,
                strides=self.strides[0],
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate[0])
        if self.rank == 2:
            outputs = K.conv2d(
                inputs,
                self.kernel,
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)
        if self.rank == 3:
            outputs = K.conv3d(
                inputs,
                self.kernel,
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            space = input_shape[1:-1]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i],
                    dilation=self.dilation_rate[i])
                new_space.append(new_dim)
            return (input_shape[0],) + tuple(new_space) + (self.filters,)
        if self.data_format == 'channels_first':
            space = input_shape[2:]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i],
                    dilation=self.dilation_rate[i])
                new_space.append(new_dim)
            return (input_shape[0], self.filters) + tuple(new_space)

    def get_config(self):
        config = {
            'rank': self.rank,
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'strides': self.strides,
            'padding': self.padding,
            'data_format': self.data_format,
            'dilation_rate': self.dilation_rate,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(_Conv, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
class ConvSN2D(Conv2D):

    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        #Spectral Normalization
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
                
        outputs = K.conv2d(
                inputs,
                W_bar,
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)
        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs
    
class ConvSN1D(Conv1D):
    
    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                 initializer=initializers.RandomNormal(0, 1),
                 name='sn',
                 trainable=False)
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True
        
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        #Spectral Normalization
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
                
        outputs = K.conv1d(
                inputs,
                W_bar,
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)
        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs

class ConvSN3D(Conv3D):    
    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True

    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        #Spectral Normalization
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
                
        outputs = K.conv3d(
                inputs,
                W_bar,
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)
        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs

        
class EmbeddingSN(Embedding):
    
    def build(self, input_shape):
        self.embeddings = self.add_weight(
            shape=(self.input_dim, self.output_dim),
            initializer=self.embeddings_initializer,
            name='embeddings',
            regularizer=self.embeddings_regularizer,
            constraint=self.embeddings_constraint,
            dtype=self.dtype)
        
        self.u = self.add_weight(shape=tuple([1, self.embeddings.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        self.built = True
        
    def call(self, inputs):
        if K.dtype(inputs) != 'int32':
            inputs = K.cast(inputs, 'int32')
            
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.embeddings.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.embeddings, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
        self.embeddings = W_bar
            
        out = K.gather(self.embeddings, inputs)
        return out 

class ConvSN2DTranspose(Conv2DTranspose):

    def build(self, input_shape):
        if len(input_shape) != 4:
            raise ValueError('Inputs should have rank ' +
                             str(4) +
                             '; Received input shape:', str(input_shape))
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (self.filters, input_dim)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        # Set input spec.
        self.input_spec = InputSpec(ndim=4, axes={channel_axis: input_dim})
        self.built = True  
    
    def call(self, inputs):
        input_shape = K.shape(inputs)
        batch_size = input_shape[0]
        if self.data_format == 'channels_first':
            h_axis, w_axis = 2, 3
        else:
            h_axis, w_axis = 1, 2

        height, width = input_shape[h_axis], input_shape[w_axis]
        kernel_h, kernel_w = self.kernel_size
        stride_h, stride_w = self.strides
        if self.output_padding is None:
            out_pad_h = out_pad_w = None
        else:
            out_pad_h, out_pad_w = self.output_padding

        # Infer the dynamic output shape:
        out_height = conv_utils.deconv_length(height,
                                              stride_h, kernel_h,
                                              self.padding,
                                              out_pad_h)
        out_width = conv_utils.deconv_length(width,
                                             stride_w, kernel_w,
                                             self.padding,
                                             out_pad_w)
        if self.data_format == 'channels_first':
            output_shape = (batch_size, self.filters, out_height, out_width)
        else:
            output_shape = (batch_size, out_height, out_width, self.filters)
            
        #Spectral Normalization    
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
        self.kernel = W_bar
        
        outputs = K.conv2d_transpose(
            inputs,
            self.kernel,
            output_shape,
            self.strides,
            padding=self.padding,
            data_format=self.data_format)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs